<a href="https://colab.research.google.com/github/mslowik777/dw_matrix_road_signs/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [17]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_signs'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1: y_train = to_categorical(y_train)
if y_test.ndim ==1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
      )
  return model


def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu',input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  
  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),


  Dense(num_classes, activation='softmax'),                    
])
  
logdir = os.path.join('logs', datetime.datetime.now().strftime("Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [13]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained,X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.5611 - accuracy: 0.3065 - val_loss: 0.8696 - val_accuracy: 0.7541
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.7556 - accuracy: 0.7648 - val_loss: 0.2025 - val_accuracy: 0.9414
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3743 - accuracy: 0.8869 - val_loss: 0.0897 - val_accuracy: 0.9764
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2537 - accuracy: 0.9239 - val_loss: 0.0540 - val_accuracy: 0.9844
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1899 - accuracy: 0.9434 - val_loss: 0.0453 - val_accuracy: 0.9865


0.964625850340136

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu',input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),


  Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),
  
  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),


  Dense(num_classes, activation='softmax'),                    
])
  
#logdir = os.path.join('logs', datetime.datetime.now().strftime("Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [26]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1837 - accuracy: 0.9646


[0.18374329805374146, 0.9646258354187012]

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])



  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose= 0,
  )
  
  score = model.evaluate(X_test, y_test,verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model':model}


In [47]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.35275375117381214, 'dropout_cnn_block_three': 0.4922008514686671, 'dropout_cnn_block_two': 0.3154813778227629, 'dropout_dense_block_one': 0.6681689917865138, 'dropout_dense_block_two': 0.41799293882764843}
accuracy=0.9156462550163269
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.33817463957839256, 'dropout_cnn_block_three': 0.4610347344959406, 'dropout_cnn_block_two': 0.3785180557926313, 'dropout_dense_block_one': 0.3978198287755651, 'dropout_dense_block_two': 0.6726123573054997}
accuracy=0.9358276724815369
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3666411872117567, 'dropout_cnn_block_three': 0.4982941562171985, 'dropout_cnn_block_two': 0.42665933135481177, 'dropout_dense_block_one': 0.6998394045711478, 'dropout_dense_block_two': 0.6925725805453851}
accuracy=0.7546485066413879
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.3685048978643783, 'dropout_cnn_block_three': 0.43918732783523856, 'dropout_cnn_block_two': 0.44441229333